# Collect github-eu-covid data

This notebook outlines preliminary findings of collecting github-eu-covid data.

Screenshots and other relevant images can be found under the `img/` directory, and can be displayed in this notebook using

    ![title](img/picture.png)

where applicable.

## Source of the data

## Collection options

Here we show different options for collection, where applicable. Any prototype code used for data collection is provided in `collect.py`.

### Option a) 

### Option b) [where applicable]

## Practical considerations
This is where we consider CPU time, financial cost, disk space requirements, and last (but not least) development time/uncertainty.

### CPU time
#### Integrated collection time
*This is an estimate of the time required to collect the data, without batching or parallelisation.*

#### Can the procedure be batched? Are there any caveats to this?

#### Real world collection time / cost
*Assume a maximum of 200 concurrent 8GB 2-core machines*

*NB (at time of writing based on [this](https://aws.amazon.com/ec2/pricing/on-demand/)) such a machine would cost $0.0944 per hour*

### Disk space (GB)

#### By entity type, estimate how many "rows" there are to collect (e.g. 100s, 1000s, etc)

#### By entity type, and based on the field types, what is the estimated disk space?

#### What does this imply for database storage costs?

### Development time
*How long do you think it will take to develop the codebase for the collection?*
*What uncertainties can you foresee?*